### L3: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

In [2]:
from crewai import Agent, Task, Crew,LLM
ollama_llm = LLM(
            model= "ollama/llama3.1:8b",#"ollama/llama3.2:latest",
            api_base="http://localhost:11434"
            )

In [3]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
    llm=ollama_llm,
	verbose=True
)

In [4]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
    llm=ollama_llm,
	verbose=True
)

###  Tools, Guardrails and Memory

In [5]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

/Users/tharhtet/.local/share/virtualenvs/Agentic_RAG_projs-hQf1ctZM/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [6]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/concepts/memory"
)

In [7]:
docs_scrape_tool

ScrapeWebsiteTool(name='Read website content', description="Tool Name: Read website content\nTool Arguments: {}\nTool Description: A tool that can be used to read https://docs.crewai.com/concepts/memory's content.", args_schema=<class 'crewai_tools.tools.scrape_website_tool.scrape_website_tool.FixedScrapeWebsiteToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x11393cd30>, result_as_answer=False, website_url='https://docs.crewai.com/concepts/memory', cookies=None, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 'Accept-Language': 'en-US,en;q=0.9', 'Referer': 'https://www.google.com/', 'Connection': 'keep-alive', 'Upgrade-Insecure-Requests': '1'})

In [8]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [9]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


In [10]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  #memory=True
)

In [11]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1069da51-4b0d-4c50-b01a-7055b534102d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 15e2eb37-fc0f-4eb7-a0e6-984178fa8093
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 15e2eb37-fc0f-4eb7-a0e6-984178fa8093
       Status: Executing Task...
    └── 🤖 Agent: Senior Support Representative
            Status: In Progress

# Agent: Senior Support Representative
## Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


🤖 Agent: Senior Support Representative
    Status: In Progress
└── 🧠 Thinking...

...

# Agent: Support Quality Assurance Specialist
## Task: Review the response drafted by the Senior Support Representative for DeepLearningAI's inquiry. Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer support.
Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly tone.
Check for references and sources used to  find the information, ensuring the response is well-supported and leaves no questions unanswered.


# Agent: Support Quality Assurance Specialist
## Final Answer: 
Dear DeepLearningAI,

Thank you for reaching out to CrewAI with your inquiry about our memory system. We appreciate your interest in integrating our technology into your projects.

To address your concerns, I've reviewed the documentation provided by our team, which covers various aspects of our memory system. Here's a detailed response that should provide all the information you need:

**1. Memory Syst

🚀 Crew: crew
├── 📋 Task: 15e2eb37-fc0f-4eb7-a0e6-984178fa8093
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
└── 📋 Task: 5b0ddf42-fa9c-4f2e-84e0-0479a3813cb7
       Status: Executing Task...
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 15e2eb37-fc0f-4eb7-a0e6-984178fa8093
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
└── 📋 Task: 5b0ddf42-fa9c-4f2e-84e0-0479a3813cb7
       Assigned to: Support Quality Assurance Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 5b0ddf42-fa9c-4f2e-84e0-0479a3813cb7                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1069da51-4b0d-4c50-b01a-7055b534102d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [12]:
from IPython.display import Markdown
Markdown(result.raw)

Dear DeepLearningAI,

Thank you for reaching out to CrewAI with your inquiry about our memory system. We appreciate your interest in integrating our technology into your projects.

To address your concerns, I've reviewed the documentation provided by our team, which covers various aspects of our memory system. Here's a detailed response that should provide all the information you need:

**1. Memory System Overview**

Our memory system is designed to provide a flexible and efficient way to manage data in your projects. It's comprised of multiple components, each with its own strengths and weaknesses, which can be configured to meet specific needs.

**2. Configuring Memory**

To get the most out of our memory system, it's essential to configure it correctly. Our documentation provides step-by-step instructions on how to set up the memory system in your project. Please refer to section 3: **Configuring Memory**, which includes detailed information on configuring User Memory, Long-Term Memory, and External Memory.

**3. Memory Components**

Our memory system consists of several key components:

*   **User Memory**: A flexible and dynamic memory component that allows you to store and retrieve data easily.
*   **Long-Term Memory**: A persistent memory component designed for storing large amounts of data over an extended period.
*   **External Memory**: An external storage component that can be used to supplement the capacity of User Memory or Long-Term Memory.

Each of these components has its own strengths and weaknesses, which are detailed in our documentation. We recommend reviewing section 4: **Memory Components** for a comprehensive understanding of each component's functionality.

**4. Memory Events**

Our memory system emits various events that can be used to monitor and debug the memory system. These events include:

*   **MemoryQueryStartedEvent**: Triggered when a query is initiated on the memory system.
*   **MemorySaveCompletedEvent**: Triggered after data has been saved successfully in the memory system.
*   **MemoryErrorEvent**: Triggered if an error occurs while interacting with the memory system.

These events can be used to track performance metrics, monitor data access patterns, and handle errors efficiently. For more information on these events and how they can be utilized, please refer to section 5: **Memory Events** in our documentation.

**Practical Applications**

Our memory system has numerous practical applications in real-world scenarios:

*   **Logging Memory Operations**: You can use the memory system to log all operations performed within it, which can aid in debugging and optimization.
*   **Tracking Performance Metrics**: By leveraging the events emitted by our memory system, you can track performance metrics such as query times, data access patterns, and storage usage.
*   **Error Handling**: Our memory system is designed with robust error handling mechanisms to ensure that your projects remain stable even in the event of errors.

For examples of how to apply these concepts in practice, please refer to section 6: **Practical Applications** in our documentation.

We hope this comprehensive response has addressed all your concerns and provided you with a thorough understanding of our memory system. If you have any further questions or require additional support, please don't hesitate to reach out to us. We're always here to help.

Thank you for choosing CrewAI!

Best regards,
\[Your Name]
Support Representative
CrewAI